In [1]:
from pprint import pprint

import numpy as np
import pandas as pd
from tqdm import tqdm

from lsh import MinHashLSH
from minhash import MinHash
import utils

In [2]:
tqdm.pandas()

/Users/mcmarten/miniconda3/envs/document-similarity/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
N = 4
K = 100

In [4]:
df = pd.read_csv('dataset/datascience-stackoverflow-questions.csv')
df['body'].replace('', np.nan, inplace=True)
df.dropna(subset=['body'], inplace=True)

In [5]:
df['text'] = df['body'].str.lower()

In [6]:
df['shingles'] = df['text'].progress_apply(lambda x: utils.generate_ngrams(x.split(), N))

100%|██████████| 51254/51254 [00:02<00:00, 23192.69it/s]


In [7]:
minhash = MinHash(num_hashes=K)

In [8]:
df = df[df["shingles"].str.len() != 0]

In [9]:
df['signature'] = df['shingles'].progress_apply(lambda x: minhash.signature(x))

100%|██████████| 51247/51247 [13:45<00:00, 62.06it/s]


In [10]:
sig_matrix = np.array(df['signature'].tolist()).T

In [11]:
minhash_lsh = MinHashLSH(df['shingles'].tolist(), signatures=sig_matrix, num_bands=20)

In [12]:
minhash_lsh.build()

In [13]:
doc_candidates = minhash_lsh.doc_candidates

In [14]:
def print_similars(doc_candidates):
    for doc_idx in doc_candidates:
        if len(doc_candidates[doc_idx]) > 0:
            print("Original document", doc_idx)
            print("TITLE:", df.iloc[doc_idx]['title'])
#             print("BODY:", df.iloc[doc_idx]['body'])
            print()
            print("All similar documents")
            similars = doc_candidates[doc_idx]
            for similar_doc_idx in similars:
                print("TITLE:", df.iloc[similar_doc_idx]['title'])
#                 print("BODY:", df.iloc[similar_doc_idx]['body'])
#                 print(df.iloc[similar_doc_idx]['text'])
            print("-" * 100)

In [15]:
print_similars(doc_candidates)

Original document 117
TITLE: nan

All similar documents
TITLE: nan
----------------------------------------------------------------------------------------------------
Original document 118
TITLE: nan

All similar documents
TITLE: nan
----------------------------------------------------------------------------------------------------
Original document 119
TITLE: nan

All similar documents
TITLE: nan
----------------------------------------------------------------------------------------------------
Original document 136
TITLE: nan

All similar documents
TITLE: nan
----------------------------------------------------------------------------------------------------
Original document 137
TITLE: nan

All similar documents
TITLE: nan
----------------------------------------------------------------------------------------------------
Original document 510
TITLE: nan

All similar documents
TITLE: nan
---------------------------------------------------------------------------------------------

TITLE: What is the Value of X in KNN and Why?

All similar documents
TITLE: logic behind weighted KNN
----------------------------------------------------------------------------------------------------
Original document 36607
TITLE: Is random forest a kind of spatial feature encoding?

All similar documents
TITLE: What is "spatial feature encoding"? Can anyone give a concrete example?
----------------------------------------------------------------------------------------------------
Original document 36665
TITLE: What is "spatial feature encoding"? Can anyone give a concrete example?

All similar documents
TITLE: Is random forest a kind of spatial feature encoding?
----------------------------------------------------------------------------------------------------
Original document 37762
TITLE: Intuitive explanation of Lovasz Softmax loss for Image Segmentation problems

All similar documents
TITLE: Can someone please explain Lovasz softmax loss? as its a bit difficult to understand 

In [16]:
idx = 2536
test_doc = df['title'].iloc[idx]
sim_docs = minhash_lsh.query(idx, 0.8)

In [17]:
test_doc

'Package for SMOTEBoost in R'

In [18]:
sim_docs

[]

In [19]:
df['title'].iloc[sim_docs].values[0]

IndexError: index 0 is out of bounds for axis 0 with size 0